## UAV-Cyber-SIM

In [1]:
import random
from collections import defaultdict

from config import Color
from helpers import clean
from mavlink.customtypes.location import ENUPose, GRAPose
from plan import Plan
from simulator import (
    QGC,
    ConfigGazebo,
    ConfigNovis,
    ConfigQGC,
    Gazebo,
    NoVisualizer,
    Simulator,
)

clean()

## Create Plans

In [2]:
gra_origin = GRAPose(lat=-35.3633280, lon=149.1652241,alt=0,heading=90) 
enu_origin = ENUPose(x=0, y=0, z=gra_origin.alt, heading=gra_origin.heading) 

gcs=[Color.RED,
     Color.ORANGE,
     Color.GREEN,
     Color.BLUE
     ]
n_uavs_per_gcs = 1
side_len = 10
altitude = 5
max_delay = 10 #sec

base_homes = ENUPose.list([
    (i * 50, j * 3 * side_len, 0, 0)
    for i in range(len(gcs))
    for j in range(n_uavs_per_gcs)
])

base_paths = [
    Plan.create_square_path(side_len=side_len, alt=altitude,heading=0) 
    for _ in base_homes
    ]

colors = [color for color in gcs for _ in range(n_uavs_per_gcs)]

msn_delays=[random.randint(0, max_delay) for _ in base_homes]


## Assign vehicles to GCS (by color)
gcs_sysids: dict[str, list[int]] = defaultdict(list)

for i,color in enumerate(colors,start=1):
    gcs_sysids[f'{color.name} {color.emoji}'].append(i)

## Gazebo Congiguration

In [3]:
gaz_config = ConfigGazebo(origin = enu_origin,
                          world_path="simulator/gazebo/worlds/runway3.world")

for path,home,c in zip(base_paths,base_homes,colors):
    gaz_config.add(base_path=path,base_home=home,color=c)
#gaz_config.show()

## QGroundControl Configuration

In [4]:
qgc_config = ConfigQGC(origin = gra_origin)

for path,home,color,delay in zip(base_paths,base_homes,colors,msn_delays):
    qgc_config.add(base_path=path,base_home=home,color=color,mission_delay=delay)
qgc_config.show()

# No Visualizer

In [5]:
novis_config = ConfigNovis(origin=gra_origin)
for home in base_homes:
    novis_config.add(base_home=home)

## Visualization Parameters

In [6]:
novis=NoVisualizer(novis_config)

gaz= Gazebo(gaz_config,gra_origin)

qgc=QGC(qgc_config)

## Launch Simulator

In [7]:
simulator = Simulator(
	visualizers=[qgc],
	gcs_sysids=gcs_sysids,
	missions=[veh.mission for veh in qgc_config.vehicles],
	terminals=['gcs'],
	verbose=1,
)

orac = simulator.launch()

🚀 GCS RED 🟥 launched (PID 363250)
🚀 GCS ORANGE 🟧 launched (PID 363251)
🚀 GCS GREEN 🟩 launched (PID 363252)
🚀 GCS BLUE 🟦 launched (PID 363253)
🔗 UAV logic 3 is connected to Oracle ⚪
🔗 UAV logic 4 is connected to Oracle ⚪
🔗 UAV logic 2 is connected to Oracle ⚪
🔗 UAV logic 1 is connected to Oracle ⚪
🗺️ QGroundControl launched for 2D visualization — simulation powered by ArduPilot SITL.


## Oracle checking

In [8]:
orac.run()

Oracle ⚪: 🏁 Starting Oracle with 4 vehicles
Oracle ⚪: ✅ Vehicle 2 completed its mission
Oracle ⚪: ✅ Vehicle 3 completed its mission
Oracle ⚪: ✅ Vehicle 4 completed its mission
Oracle ⚪: ✅ Vehicle 1 completed its mission
